# OFB Mode (Output Feedback Mode)

OFB mode is a block cipher mode of operation that effectively turns a block cipher (e.g. AES) into a stream cipher. In OFB mode, a keystream is generated by repeatedly encrypting an initialization vector (IV) or the previous output, and then XORed with the plaintext to produce the ciphertext. Encryption and decryption are performed using the exact same procedure, since XOR is its own inverse.

---

## How OFB Mode Works

1. **Initialization:**
   - Begin with a random IV.
   - Compute the first keystream block:  
     `O₁ = E_K(IV)`  
     where `E_K` denotes encryption with key `K`.

2. **Keystream Generation:**
   - For each subsequent block, compute:  
     `Oᵢ = E_K(Oᵢ₋₁)`  
     Continue this process to generate as many keystream blocks as needed.

3. **Encryption / Decryption:**
   - **Encryption:**  
     For each plaintext block `Pᵢ`, compute  
     `Cᵢ = Pᵢ ⊕ Oᵢ`
   - **Decryption:**  
     For each ciphertext block `Cᵢ`, recover the plaintext as  
     `Pᵢ = Cᵢ ⊕ Oᵢ`

   Since XOR is reversible, the same process performs both encryption and decryption.

---

## Communication Flow

```text
Alice                             Bob                           Eve
-----                             ---                           ---
Plaintext P
   |--- Encrypt using OFB with key K and IV --->
   |             Producing Ciphertext C
   |                                              
   |--- Decrypt using the same key K and IV ---> Plaintext P
                                            
(Eve observes C, but without knowing K, she cannot reconstruct the keystream)
```

- **Note:** The IV must be unique and unpredictable for every encryption. Reusing the same (K, IV) pair completely compromises security because it leads to keystream reuse.

---

## Key Parameters

- **Key (K):**  
  AES-128, AES-192, or AES-256 key used for encryption.

- **IV (Initialization Vector):**  
  Must be random and unique for each encryption session. For AES, the IV is typically 128 bits (16 bytes).

- **Block Size:**  
  Determined by the underlying cipher (e.g., AES uses 128-bit blocks).

---

## Security Considerations

- **Advantages:**
  - **No Padding Required:**  
    Works like a stream cipher; partial blocks can be encrypted without padding.
  - **Error Propagation:**  
    A bit error in the ciphertext affects only the corresponding plaintext bits after decryption (no error propagation across blocks).

- **Drawbacks:**
  - **IV Reuse Catastrophe:**  
    Reusing a (K, IV) pair leads to reuse of the keystream, making the encryption completely insecure.
  - **No Data Integrity:**  
    OFB does not provide any authenticity or integrity guarantees. It is recommended to combine OFB with a Message Authentication Code (MAC) or use an AEAD mode (like AES-GCM) for authenticated encryption.

- **Usage Considerations:**  
  Although OFB sees occasional use in legacy systems, modern systems often prefer CTR mode due to its parallelizability and ease of use.

---
## Encryption 

![OFB Encryption](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b0/OFB_encryption.svg/601px-OFB_encryption.svg.png)
---

## Decryption

![OFB Decryption](https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/OFB_decryption.svg/601px-OFB_decryption.svg.png)

---

## Practical Applications

- **Legacy Communication Protocols:**  
  OFB was historically implemented in legacy systems.
- **Stream-Like Requirements:**  
  Situations where a stream cipher behavior is desired without block padding.

---
---

**Summary:**  
OFB mode transforms a block cipher into a stream cipher by generating a keystream from a continuously updated feedback value. It provides efficient encryption without padding and limited error propagation, but always ensure that (K, IV) is never reused, and combine it with integrity checks (e.g., MAC) if data authenticity is required.

In [2]:
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes

# --- Key & IV ---
key = get_random_bytes(16)   # AES-128
iv = get_random_bytes(16)    # 16-byte IV

# --- Encrypt ---
cipher_enc = AES.new(key, AES.MODE_OFB, iv=iv)

plaintext = b"OFB mode: stream-like encryption"
ciphertext = cipher_enc.encrypt(plaintext)

print("Ciphertext (hex):", ciphertext.hex())

# --- Decrypt ---
cipher_dec = AES.new(key, AES.MODE_OFB, iv=iv)
recovered = cipher_dec.decrypt(ciphertext)

print("Recovered:", recovered)


Ciphertext (hex): c79b406eba2b1a703e7048fd484e107cd0dd23d9bc28e0cc7240d4981382ad83
Recovered: b'OFB mode: stream-like encryption'
